In [5]:
#This notebook is implemented using google earth engine, an account authentication and initialization will be required!
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()


Enter verification code: 4/1AX4XfWgvlcP6xkIWFIZeXr2A1zyC7dbQliANOOiAy1NY6p1aPx-rdltUA8E

Successfully saved authorization token.


In [7]:
# Import the Folium library.
import folium


In [8]:
geoJSON= {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              32.6513671875,
              -24.746831298412033
            ],
            [
              32.36572265625,
              -25.19500042430748
            ],
            [
              32.71728515625,
              -25.552353652165465
            ],
            [
              33.15673828125,
              -24.95618002005594
            ],
            [
              32.6513671875,
              -24.746831298412033
            ]
          ]
        ]
      }
    }
  ]
}

In [9]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

In [398]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [373]:
# Add custom base maps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

In [374]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [33]:
vis_params = {'min': 0, 'max': 1, 'palette': 
['red', 'yellow','green']}

In [34]:
vis_params = {'min': 0, 'max': 1, 'palette': 
['red', 'yellow','green']}
basemaps = {
'Google Maps': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Maps',
overlay = True,
control = True
),
'Google Satellite': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Satellite',
overlay = True,
control = True
)}
def add_ee_layer(self, ee_object, vis_params, name):
 try:
  if isinstance(ee_object, ee.image.Image):
   map_id_dict = ee.Image(ee_object).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.imagecollection.ImageCollection):
   ee_object_new = ee_object.mosaic()
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name, 
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.geometry.Geometry):
   folium.GeoJson(
   data = ee_object.getInfo(),
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object,ee.featurecollection.FeatureCollection):
   ee_object_new = ee.Image().paint(ee_object, 0, 2)
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

 except:
  print("Could not display {}".format(name))

In [50]:
def clip_img(img):
    """Clips a list of images."""
    return ee.Image(img).clip(aoi)

im_list = im_coll.toList(im_coll.size())
im_list = ee.List(im_list.map(clip_img))

im_list.length().getInfo()

81

In [51]:
#Function to select only th vv bands
def selectvv(current):
    return ee.Image(current).select('VV')

vv_list = im_list.map(selectvv)

location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=11)
vv_scence = (ee.Image.rgb(vv_list.get(10), vv_list.get(11), vv_list.get(12))
              .log10().multiply(10))
#mp.add_ee_layer(vv_scence, {'min': -20,'max': 0}, 'rgb composite')
#mp.add_child(folium.LayerControl())

#Function to select only th vh bands and convert them to list
def selectvh(current):
    return ee.Image(current).select('VH')

vh_list = im_list.map(selectvh)

In [386]:
#SAR band-C collection for crop identification using the backscatter value from the two polarizations VV and VH
im_col = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterBounds(aoi)
           .filterDate(ee.Date('2020-02-01'),ee.Date('2020-12-30'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')))
           #.filter(ee.Filter.eq('relativeOrbitNumber_start', 154))
           #.map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           #.sort('date'))
##Compute and print the number of image available per km2, area of the polygon and the number of images in the collection.
print("The number of image is ",im_col.size().getInfo())

The number of image is  184


In [387]:
# scene as captured by single polarization vv
location = aoi.centroid().coordinates().getInfo()[::-1]
vvFirst = folium.Map(location=location, zoom_start=10)
vv1 = ee.Image(im_col.first()).select('VV').clip(aoi)
vvFirst.add_ee_layer(vv1, {'bands':'VV','min': -20,'max': 0}, 'first image')
vvFirst.add_child(folium.LayerControl())

In [388]:
# scene as captured by single polarization vv
location = aoi.centroid().coordinates().getInfo()[::-1]
vhFirst = folium.Map(location=location, zoom_start=10)
vh1 = ee.Image(im_col.first()).select('VH').clip(aoi)
vhFirst.add_ee_layer(vh1, {'bands':'VH','min': -20,'max': 0}, 'first image')
vhFirst.add_child(folium.LayerControl())

In [389]:
#RGB composite 
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(im_col.first().select('VV'),
                   im_col.first().select('VH'),
                   im_col.first().select('VV').divide(im_col.first().select('VH')))

# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'RGB')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [390]:
#####Apply a filter to correct the speckels

# Filter speckle noise
def filterSpeckles(img):
    vv = img.select('VV')
    vv_smoothed = vv.focal_median(100,'circle','meters').rename('VV_Filtered')
    return img.addBands(vv_smoothed)
#Map speckle noise filter across collection. Result is same collection, with smoothed VV band added to each image
S1 = im_col.map(filterSpeckles)


In [391]:
location = aoi.centroid().coordinates().getInfo()[::-1]
img1_filt = ee.Image(S1.first()).select('VV_Filtered').clip(aoi)
vvFilt = folium.Map(location=location, zoom_start=10)
vvFilt.add_ee_layer(img1_filt, {'min': -20,'max': 0}, 'filtered image')
vvFilt.add_child(folium.LayerControl())

Now we classify the tobacco pixels in the SAR images. We will use a simple threshold approach to identify the plant. We will pick a threshold of backscatter and classify all pixels below that threshohld as Tobacco. Emperical evidence suggest that tobacco has a backscatter value in the randge of -17.876 to -19.235 to give us a ballpark estimate of the tobacco area. This threshold is simply based on a similar research done, try adjusting the threshold and see how your classification changes. Optimal thesholds can be determined through comparison with a training dataset, or through more robust statstical techniques. This is a first approach

In [392]:
### Classifier built around single polarization vv
###Use the backscatter values of sar image to determine crops that are tobacco. We give threshold of tobacco such that any backscatter
#less than the threshold is
def classifier(img):
    vv = img.select('VV_Filtered')
    Tobacco =vv.gt(-19.235).And(vv.lt(-17.876)).rename('Tobacco') #pick all pixels below threshold and set them equal to 1. All other pixels set to 0
    Tobacco = Tobacco.updateMask(Tobacco) #Remove all pixels equal to 0
    return img.addBands(Tobacco)#Return image with added classified tobacco band

vv_classifier= S1.map(classifier)


In [ ]:
u

In [393]:
#Make time series of tobacco pixels within region
ClassChart = ui.chart.Image.series(**{'imageCollection': vv_classifier.select('Tobacco'),'region': aoi, 'reducer': ee.Reducer.sum(),'scale': 100})#.setOptions({title: 'Inundated Pixels', hAxis: {'title': 'Date'}, vAxis: {'title': 'Number of Inundated Pixels'},lineWidth: 2}))

#Set the postion of the chart and add it to the map    
#ClassChart.style().set({
 #   position: 'bottom-right',
  #  width: '500px',
   # height: '300px'
  #});
  
#Map.add(ClassChart)
ClassChart.renderWidget(width='100%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [397]:
#Adding the classification toidentify tobacco plants on the  map, Green pacthes indicates presensce of tobacco
visParams = {min: 0,max: 1,'palette': ['#FFFFFF','#17d868']}
vv_map= folium.Map(location=aoi.centroid().coordinates().getInfo()[::-1],zoom_start=10)  
vv_map.add_ee_layer( vv_classifier.select('Tobacco').first().clip(aoi), visParams, 'Tobacco sites')
vv_map.add_child(folium.LayerControl())

In [264]:
#####Apply a filter to correct the speckels

# Filter speckle noise
def filterSpeckles(img):
    v_h = img.select('VH')
    vhsmoothed = v_h.focal_median(100,'circle','meters').rename('VH_Filtered')
    return img.addBands(vhsmoothed)
#Map speckle noise filter across collection. Result is same collection, with smoothed VV band added to each image
S_vh = im_col.map(filterSpeckles)


In [265]:
### Classifier built around single polarization vh
###Use the backscatter values of sar image to determine crops that are tobacco. We give threshold of tobacco such that any backscatter
#less than the threshold is
def classifier(img):
    v_h = img.select('VH_Filtered')
    Tobacco = v_h.gt(-19.235).And( v_h.lt(-17.876)).rename('Tobacco') #pick all pixels below threshold and set them equal to 1. All other pixels set to 0
    Tobacco = Tobacco.updateMask(Tobacco) #Remove all pixels equal to 0
    return img.addBands(Tobacco)#Return image with added classified tobacco band

vhclassifier= S_vh.map(classifier)

In [384]:
#Make time series of tobacco pixels within region with vh
ClassChart = ui.chart.Image.series(**{'imageCollection': vhclassifier.select('Tobacco'),'region': aoi, 'reducer': ee.Reducer.sum(),'scale': 100, 'xProperty': 'system:time_start'})#.setOptions({title: 'Inundated Pixels', hAxis: {'title': 'Date'}, vAxis: {'title': 'Number of Inundated Pixels'},lineWidth: 2}))

#Set the postion of the chart and add it to the map    
#ClassChart.style().set({
 #   position: 'bottom-right',
  #  width: '500px',
   # height: '300px'
  #});
  
#Map.add(ClassChart)
ClassChart.renderWidget(width='100%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [222]:
#Plot of pixels meeting the backscater requirement to identify the locations of tobacco plant using the vh single polarisation
vh_map= folium.Map(location=aoi.centroid().coordinates().getInfo()[::-1], zoom_start = 10)  
vh_map.add_ee_layer( vhclassifier.select('Tobacco').first(), visParams, 'Tobacco sites')
basemaps['Google Maps'].add_to(vh_map)#'Google Satellite'
#Green patches indicates the presence of tobacco
vh_map.add_child(folium.LayerControl())


In [267]:
### Classifier built around co polarization vh and vv
def classifier(img):
    vvvh = img.select('VV').add(img.select('VH'))
    Tobacco = vvvh.gt(-19.235).And(vvvh.lt(-17.876)).rename('Tobacco') #pick all pixels below threshold and set them equal to 1. All other pixels set to 0
    Tobacco = Tobacco.updateMask(Tobacco) #Remove all pixels equal to 0
    return img.addBands(Tobacco)#Return image with added classified tobacco band
vvvh_classifier= im_col.map(classifier)


In [359]:
#Make time series of tobacco pixels within region with vh
ClassChart = ui.chart.Image.series(**{'imageCollection': vvvh_classifier.select('Tobacco'),'region': aoi, 'reducer': ee.Reducer.sum(),'scale': 100,'maxPixels':1e9, 'bestEffort':True})#.setOptions({title: 'Inundated Pixels', hAxis: {'title': 'Date'}, vAxis: {'title': 'Number of Inundated Pixels'},lineWidth: 2}))

#Set the postion of the chart and add it to the map    
#ClassChart.style().set({
 #   position: 'bottom-right',
  #  width: '500px',
   # height: '300px'
  #});
  
#Map.add(ClassChart)
ClassChart.renderWidget(width='100%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [269]:
#Plot of pixels meeting the backscater requirement to identify the locations of tobacco plant using the cross vh$vv  polarisation
vvvh_map= folium.Map(location=aoi.centroid().coordinates().getInfo()[::-1], zoom_start = 10)  
vvvh_map.add_ee_layer(vvvh_classifier.select('Tobacco').first(), visParams, 'Tobacco sites')
#vh_map.add_ee_layer( vh_classifier.select('Water').first(), visParams, 'Water')
vvvh_map.add_child(folium.LayerControl())

An insight into the availability of images at given location with varying sizes. We filter on varrying dates to have a fair idea of their availability. We start by having an annual collection, then we have the collection for half a year(6 months) and into quarters.

In [76]:
#Create a multi location points with individual point constituents
multi =  ee.Geometry.MultiPoint([
            [32.6513671875,-24.746831298412033],
            [32.36572265625,-25.19500042430748],
            [32.71728515625,-25.552353652165465],
            [33.15673828125,-24.95618002005594],
            [32.6513671875, -24.746831298412033]])
geometries = multi.geometries()

# Divide the entire polygon into smaller
aoi_1 = geometries.get(0)
aoi_2 = geometries.get(1)
aoi_3 = geometries.get(2)
aoi_4 = geometries.get(2)

In [254]:
#SAR band-C annual availability of images for the aoi
Annualcollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterBounds(aoi)
           .filterDate(ee.Date('2020-01-01'),ee.Date('2020-12-31'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
           #.filter(ee.Filter.eq('relativeOrbitNumber_start', 154))
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))
##Compute and print the number of image available per km2, area of the polygon and the number of images in the collection.
print("The number of image is ",Annualcollection.size().getInfo())

area = ee.Image.pixelArea().divide(1000000)
area2 = area.reduceRegion(**{'reducer': ee.Reducer.sum(), 'geometry':aoi,'scale':30, 'maxPixels':1E13})
print('Area of point in km2 is', ee.Number(area2.get('area')).getInfo() )
print('Number of km2 covered by an image', ee.Number(area2.get('area')).getInfo()/Annualcollection.size().getInfo())

The number of image is  201
Area of point in km2 is 3648.9721660782807
Number of km2 covered by an image 18.15409037849891


In [253]:
Half_annual = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterBounds(aoi)
           .filterDate(ee.Date('2020-01-01'),ee.Date('2020-06-30'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
           #.filter(ee.Filter.eq('relativeOrbitNumber_start', 154))
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))
##Compute and print the number of image available per km2, area of the polygon and the number of images in the collection.
print("The number of image is ",Half_annual.size().getInfo())

area = ee.Image.pixelArea().divide(1000000)
area2 = area.reduceRegion(**{'reducer': ee.Reducer.sum(), 'geometry':aoi,'scale':30, 'maxPixels':1E13})
print('Area of point in km2 is', ee.Number(area2.get('area')).getInfo() )
print('Number of km2 covered by an image', ee.Number(area2.get('area')).getInfo()/Half_annual.size().getInfo())

The number of image is  97
Area of point in km2 is 3648.9721660782807
Number of km2 covered by an image 37.61826975338434


In [252]:
SecondHalf_annual = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterBounds(aoi)
           .filterDate(ee.Date('2020-07-01'),ee.Date('2020-12-31'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
           #.filter(ee.Filter.eq('relativeOrbitNumber_start', 154))
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))
##Compute and print the number of image available per km2, area of the polygon and the number of images in the collection.
print("The number of image is ",SecondHalf_annual.size().getInfo())

area = ee.Image.pixelArea().divide(1000000)
area2 = area.reduceRegion(**{'reducer': ee.Reducer.sum(), 'geometry':aoi,'scale':30, 'maxPixels':1E13})
print('Area of point in km2 is', ee.Number(area2.get('area')).getInfo() )
print('Number of km2 covered by an image', ee.Number(area2.get('area')).getInfo()/SecondHalf_annual.size().getInfo())

The number of image is  103
Area of point in km2 is 3648.9721660782807
Number of km2 covered by an image 35.426914233769715


In [79]:
### Classifier built around single polarization vh
##def classifier(img):
  ##  vh = img.select('VV')
   ## Water =vh.lt(-20).rename('Water') #pick all pixels below threshold and set them equal to 1. All other pixels set to 0
    ##Water = Water.updateMask(Water) #Remove all pixels equal to 0
    ##return img.addBands(Water)#Return image with added classified tobacco band
##wh_classifier= im_col.map(classifier)


In [80]:
vis = {min: 0,max: 1,'palette': ['#FFFFFF','#a5abeb']}

In [224]:
#Plot of pixels meeting the backscater requirement to identify the locations of tobacco plant using the cross vh$vv  polarisation
vvvh_map= folium.Map(location=aoi.centroid().coordinates().getInfo()[::-1], zoom_start = 10)  
vvvh_map.add_ee_layer( vvvh_classifier.select('Tobacco').first(), visParams, 'Tobacco sites')
#vvvh_map.add_ee_layer( wh_classifier.select('Water').first(), vis, 'Potatoes')

#vh_map.add_ee_layer( vh_classifier.select('Water').first(), visParams, 'Water')
vvvh_map.add_child(folium.LayerControl())

Using shapefiles together with geojson files to create an interative map. Session still in progress

In [352]:
import os
import json
from ipyleaflet import GeoJSON

In [328]:
country_shp = (r'C:\Users\bright\Desktop\aim_ishango\data\gadm36_MOZ_shp\gadm36_MOZ_1.shp')

In [329]:
country = geemap.shp_to_ee(country_shp)

In [350]:
file_path = os.path.abspath(r'C:/Users/bright/Desktop/aim_ishango/data/map.geojson')      

with open(file_path) as f:
    json_data = json.load(f)

In [337]:
Map.addLayer(country,{}, 'Country')
json_layer = GeoJSON(data=json_data, name='AOI', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
Map.add_layer(json_layer)

In [354]:
Map

Map(bottom=9257.0, center=[-16.573022719182777, 36.672601535591305], controls=(WidgetControl(options=['positio…

In [ ]:
#####Apply a filter to correct the speckels

# Filter speckle noise
#def filterSpeckles(img):
 #   vv = img.select('VV')
  #  vh = img.select('VH')
   # vv_smoothed = vv.focal_median(100,'circle','meters').rename('VV_Filtered')
   # return img.addBands(vv_smoothed)
#Map speckle noise filter across collection. Result is same collection, with smoothed VV band added to each image
#S1 = im_col.map(filterSpeckles)
